# Week 3 Assignment

Install `pip3 install statsmodels`

In [54]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import edhec_risk_kit_207 as erk # created by JLC to add functions from lab_23.ipynb
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load priors as suggested on course

Load the 49 industries Value weighted returns and cap weights, and use the period 2013-2018 both included. For the period, use the starting cap weights of the period. Limit yourself to the following 5 industry sectors: 'Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food'. You will need to compute the correlation matrix as well as the volatilities. (Hint: Remember to annualize the volatilities by multiplying the volatility you get from the monthly data by the sqrt of 12)

In [55]:
ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":]
ind49_mcap = erk.get_ind_market_caps(49, weights=True)["2013":]
inds = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
# Correlation matrix
rho_ = ind49_rets[inds].corr()
# volatilities
vols_ = (ind49_rets[inds].std()*np.sqrt(12))
# priors
sigma_prior_ =  (vols_.T).dot(vols_) * rho_
sigma_prior_

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.108718,0.057064,0.070764,0.062429,0.040410
Fin,0.057064,0.108718,0.084545,0.064723,0.037292
Whlsl,0.070764,0.084545,0.108718,0.080859,0.058843
Rtail,0.062429,0.064723,0.080859,0.108718,0.062519
Food,0.040410,0.037292,0.058843,0.062519,0.108718


## 1. Question 1

Using the same value of delta used in the He-Litterman paper of 2.5 and using the same sigma prior methodology used in the notebook and in the paper, compute the implied returns vector.

In [56]:
# pd.Series([.44, .56], index=['INTC', 'PFE']) # shows the shape
cw = erk.weight_cw(ind49_rets[inds], cap_weights=ind49_mcap[inds])
cw

Hlth     0.042624
Fin      0.184533
Whlsl    0.096043
Rtail    0.537344
Food     0.139456
Name: 2013-02, dtype: float64

In [57]:
pi = erk.implied_returns(delta=2.5, sigma=sigma_prior_, w=cw)
pi

Hlth     0.152854
Fin      0.176483
Whlsl    0.201786
Rtail    0.223770
Food     0.157528
Name: Implied Returns, dtype: float64

Which industry sector has the highest capweight?
Enter your answer as text, exactly as they are named in the Data file (i.e. 'Hlth', 'Fin', 'Whlsl', 'Rtail', or 'Food')

In [58]:
cw.idxmax() # Answer is Rtail

'Rtail'

## 2. Question 2
Use the same data as the previous question, which industry sector has the highest implied return? 
Enter your answer as text, exactly as they are named in the Data file (i.e. 'Hlth', 'Fin', 'Whlsl', 'Rtail', or 'Food')

In [59]:
pi.idxmax() # Answer is Rtail

'Rtail'

## 3. Question 3
Use the same data and assumptions as the previous question.
Which industry sector has the lowest implied return?    
Enter your answer as text, exactly as they are named in the Data file (i.e. 'Hlth', 'Fin', 'Whlsl', 'Rtail', or 'Food')

In [60]:
pi.idxmin() # Answer is Hlth

'Hlth'

## 4. Question 4
Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)
What is the entry you will use for the Pick Matrix P for Whlsl. (Hint: Remember to use the correct sign)
Enter the number as a positive or negative number correct to at least two decimal places (e.g. -0.23 or +0.23)

In [61]:
# Hlth will outperform Rtail and Whlsl by 3%
q = pd.Series([.03]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(inds), index=inds).T
# find the relative market caps of Rtail and Whlsl to split the relative outperformance of Hlth ...
w_Rtail =  cw.loc["Rtail"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
w_Whlsl =  cw.loc["Whlsl"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_Rtail
p.iloc[0]['Whlsl'] = -w_Whlsl

In [62]:
p.round(2)['Whlsl'] # Answer is -0.15

0   -0.15
Name: Whlsl, dtype: float64

## 5. Question 5
Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)
What is the entry you will use for the Pick Matrix P for Rtail. (Hint: Remember to use the correct sign)
Enter the number as a positive or negative number correct to at least two decimal places (e.g. -0.234 or +0.234)

In [63]:
p.round(2)['Rtail'] # Answer is -0.85

0   -0.85
Name: Rtail, dtype: float64

## 6. Question 6
Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)
Once you impose this view (use delta = 2.5 and tau = 0.05 as in the paper), which sector has the **lowest implied return**?
Enter your answer as text, exactly as they are named in the Data file (i.e. 'Hlth', 'Fin', 'Whlsl', 'Rtail', or 'Food')

In [64]:
delta = 2.5
tau = 0.05
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = erk.bl(w_prior=cw, sigma_prior=sigma_prior_, p=p, q=q, tau = tau)
# Black Litterman Implied Mu
bl_mu

Hlth     0.179360
Fin      0.170206
Whlsl    0.193356
Rtail    0.199007
Food     0.144841
dtype: float64

In [65]:
bl_mu.idxmin() # Answer is Food

'Food'

## 7. Question 7
Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)
Which sector now has the **highest weight** in the MSR portfolio using the Black-Litterman model?
Enter your answer as text, exactly as they are named in the Data file (i.e. 'Hlth', 'Fin', 'Whlsl', 'Rtail', or 'Food')

In [66]:
wstar = erk.w_star(delta=delta, sigma=bl_sigma, mu=bl_mu)
wstar

Hlth     0.262745
Fin      0.175746
Whlsl    0.057784
Rtail    0.323291
Food     0.132815
dtype: float64

In [67]:
wstar.idxmax() # Answer is Rtail

'Rtail'

## 8. Question 8
Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)
Which sector now has the **lowest weight** in the MSR portfolio using the Black-Litterman model?
Enter your answer as text, exactly as they are named in the Data file (i.e. 'Hlth', 'Fin', 'Whlsl', 'Rtail', or 'Food')

In [68]:
wstar.idxmin() # Answer is Whlsl

'Whlsl'

## 9. Question 9
Now, let’s assume you change the relative view. You still think that it Hlth will outperform Rtail and Whlsl but you think that the outperformance will be 5% not the 3% you originally anticipated.
Which of the arrays will you need to update?    
- P and not Q 
- Q and not P  
- Both P and Q  
- Neither P nor Q but a different parameter

In [69]:
# Answer is Q and not P

## 10. Question 10
Now, let’s assume you change the relative view. You still think that it Hlth will outperform Rtail and Whlsl but you think that the outperformance will be 5% not the 3% you originally anticipated.
Under this new view which sector has the **highest expected return**?
Enter your answer as text, exactly as they are named in the Data file (i.e. 'Hlth', 'Fin', 'Whlsl', 'Rtail', or 'Food')

In [70]:
# Hlth will outperform Rtail and Whlsl by 5%
q2 = pd.Series([.05])
# Find the Black Litterman Expected Returns
bl_mu2, bl_sigma2 = erk.bl(w_prior=cw, sigma_prior=sigma_prior_, p=p, q=q2, tau = tau)
# Black Litterman Implied Mu
bl_mu2.idxmax() # Answer is Rtail

'Rtail'

## 11. Question 11
Now, let’s assume you change the relative view. You still think that it Hlth will outperform Rtail and Whlsl but you think that the outperformance will be 5% not the 3% you originally anticipated.
Under this new view which sector does the Black-Litterman model assign the **highest weight**?
Enter your answer as text, exactly as they are named in the Data file (i.e. 'Hlth', 'Fin', 'Whlsl', 'Rtail', or 'Food')

In [71]:
wstar2 = erk.w_star(delta=delta, sigma=bl_sigma2, mu=bl_mu2)
wstar2.idxmax() # Answer is Hlth

'Hlth'